<a href="https://colab.research.google.com/github/ravindrabharathi/tf_utils/blob/active_learning_drought_watch/test/droughtwatch_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization,Concatenate,Lambda,Activation,Input,Dropout
 
from tensorflow.keras.optimizers import Adam,SGD
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow.data import Dataset

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

import math

In [2]:
!pip install wandb -qqq
import wandb
wandb.login()


     |████████████████████████████████| 2.1MB 14.5MB/s 
     |████████████████████████████████| 133kB 51.1MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 174kB 59.5MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
run=wandb.init(name='draughtwatch_12', 
           project='Wandb_Drought_Watch',
           notes='Drought Watch dataset with Bands B2-B7, custom ResNet', 
           tags=['DroughtWatch', 'tf_utils','ResNet'])

wandb: Currently logged in as: ravindra (use `wandb login --relogin` to force relogin)


In [4]:
!pip install --upgrade git+https://github.com/ravindrabharathi/tf_utils@active_learning_drought_watch

  Cloning https://github.com/ravindrabharathi/tf_utils (to revision active_learning_drought_watch) to /tmp/pip-req-build-4lv_11g3
  Running command git clone -q https://github.com/ravindrabharathi/tf_utils /tmp/pip-req-build-4lv_11g3
  Running command git checkout -b active_learning_drought_watch --track origin/active_learning_drought_watch
  Switched to a new branch 'active_learning_drought_watch'
  Branch 'active_learning_drought_watch' set up to track remote branch 'active_learning_drought_watch' from 'origin'.
  Created wheel for tf-utils: filename=tf_utils-0.4-cp37-none-any.whl size=8996 sha256=650dda2d57b91b12b77a2b07590cea816ccf03cbedb401345d6db673ca378732
  Stored in directory: /tmp/pip-ephem-wheel-cache-ejopt_55/wheels/80/a8/35/f3e2a85eff1b8cb7e5e54cbd69a209356cf2153f9a4ee67904
Successfully built tf-utils


In [5]:
import tf_utils.data as ds 
import tf_utils.visualize as vz
import tf_utils.transform as tfm

In [6]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


# New Section

In [7]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [8]:
dirlist = lambda di: [os.path.join(di, file) for file in os.listdir(di) if 'part-' in file]
training_files = dirlist('/gdrive/MyDrive/wandb-dw/data/droughtwatch_data/train/')
val_files = dirlist('/gdrive/MyDrive/wandb-dw/data/droughtwatch_data/val/')



In [9]:
train_ds=ds.get_train_ds(training_files,batch_size=ds.batch_size,shuffle=True,distort=True,distort_fn=tfm.aug1)
#unlabelled_ds=ds.get_unlabelled_ds()
test_ds=ds.get_test_ds(val_files)

distorting...
Finished 'get_tf_dataset_2' in 3.7929 secs
Finished 'get_tf_dataset_in_batches' in 3.7933 secs
Finished 'get_train_ds' in 3.7935 secs
Finished 'get_tf_dataset_2' in 0.0763 secs
Finished 'get_tf_dataset_in_batches' in 0.0766 secs
Finished 'get_test_ds' in 0.0774 secs


In [10]:
'''
for idx, row in enumerate(train_ds):
    print (row)
    if idx > 1:
        break

'''        

'\nfor idx, row in enumerate(train_ds):\n    print (row)\n    if idx > 1:\n        break\n\n'

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D,  Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.optimizers import Adam


  

In [12]:
'''
def conv(inp,f=32,k=3):
  conv_layer=Conv2D(f,k,use_bias=False,padding='same')(inp)
  conv_layer=BatchNormalization()(conv_layer)
  conv_layer=Activation('relu')(conv_layer)
  return conv_layer
'''  

"\ndef conv(inp,f=32,k=3):\n  conv_layer=Conv2D(f,k,use_bias=False,padding='same')(inp)\n  conv_layer=BatchNormalization()(conv_layer)\n  conv_layer=Activation('relu')(conv_layer)\n  return conv_layer\n"

In [13]:
import math
WEIGHT_DECAY=1.25e-4
reg=tf.keras.regularizers.l2(WEIGHT_DECAY)
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

def conv(inp,f=32,k=3):
  conv_layer=Conv2D(f,k,use_bias=False,padding='same',kernel_initializer=init_pytorch, kernel_regularizer=reg)(inp)
  conv_layer=BatchNormalization(momentum=0.9, epsilon=1e-5)(conv_layer)
  conv_layer=Activation('relu')(conv_layer)
  return conv_layer
def resBlk(inp,f=32,k=3,residual=True) :
  res1=conv(inp,f,k)
  res1=MaxPooling2D(pool_size=(2,2))(res1)
  if residual:
    res2=conv(res1,f,k)
    res3=conv(res2,f,k)
    return res1+res3
  else:
    return res1  

In [14]:
def build_model():
  f=64
  inp=Input(shape=(65,65,6))
  layer1=conv(inp,f,3)
  res1=resBlk(layer1,f*2,3)
  
  res2=resBlk(res1,f*4,3,False)
  
  res3=resBlk(res2,f*8,3)

  #res4=resBlk(res3,f*8,3,False)

  #res5=resBlk(res4,f*8,3)
  
  
  layer2=tf.keras.layers.GlobalMaxPooling2D()(res3)
  layer3=tf.keras.layers.Dense(4, kernel_initializer=init_pytorch, use_bias=False,kernel_regularizer=reg)(layer2)
  layer4=Lambda(lambda x: x*0.125)(layer3)
  out=Activation('softmax')(layer4)
  model=tf.keras.models.Model(inputs=[inp],outputs=[out])
  model.summary()
  return model 

In [15]:
model=build_model()
opt=SGD(lr=0.0001,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,metrics=['accuracy']
              )

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 65, 65, 6)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 65, 65, 64)   3456        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 65, 65, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 65, 65, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [16]:
class Log2wandb_Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    wandb.log({
        "Epoch": epoch,
        "Train Loss": logs["loss"],
        "Train Acc": logs["accuracy"],
        "Val Loss": logs["val_loss"],
        "val_acc": logs["val_accuracy"],
        "LR":model.optimizer.lr.numpy()
        
        })
        

In [17]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_schedule():
    
    def schedule(epoch):
      
      lr=np.interp([epoch],[0, EPOCHS//5,EPOCHS//2,EPOCHS], [0.0001, 0.1, 0.006,0.001])[0]
      print('epoch ', epoch+1, ': setting learning rate to ',lr)
      return lr
    
    return LearningRateScheduler(schedule)

lr_sched = lr_schedule()

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_cpt=ModelCheckpoint(filepath='/gdrive/MyDrive/wandb-dw/best_model12.h5', 
                          verbose=1, save_best_only=True,monitor='val_accuracy',mode='auto')

In [19]:
import numpy as np
batch_size=128
EPOCHS=20
callback_list=[Log2wandb_Callback(),lr_sched,model_cpt]
model.fit(train_ds,epochs=EPOCHS, steps_per_epoch=np.ceil(86317/batch_size), 
          validation_data=test_ds, validation_steps=np.ceil(10778/batch_size),
          callbacks=callback_list,
          verbose=1)

Epoch 1/20
epoch  1 : setting learning rate to  0.0001
675/675 [==============================] - 421s 591ms/step - loss: 1.1687 - accuracy: 0.5818 - val_loss: 1.2422 - val_accuracy: 0.5973

Epoch 00001: val_accuracy improved from -inf to 0.59733, saving model to /gdrive/MyDrive/wandb-dw/best_model12.h5
Epoch 2/20
epoch  2 : setting learning rate to  0.025075
675/675 [==============================] - 49s 73ms/step - loss: 1.0192 - accuracy: 0.6304 - val_loss: 1.0360 - val_accuracy: 0.6445

Epoch 00002: val_accuracy improved from 0.59733 to 0.64449, saving model to /gdrive/MyDrive/wandb-dw/best_model12.h5
Epoch 3/20
epoch  3 : setting learning rate to  0.050050000000000004
675/675 [==============================] - 49s 73ms/step - loss: 0.9202 - accuracy: 0.6781 - val_loss: 1.1871 - val_accuracy: 0.5255

Epoch 00003: val_accuracy did not improve from 0.64449
Epoch 4/20
epoch  4 : setting learning rate to  0.07502500000000001
675/675 [==============================] - 49s 72ms/step - lo

In [23]:
def lr_schedule():
    
    def schedule(epoch):
      
      lr=np.interp([epoch],[20,25,EPOCHS], [0.001,0.006, 0.0001])[0]
      print('epoch ', epoch+1, ': setting learning rate to ',lr)
      return lr
    
    return LearningRateScheduler(schedule)

lr_sched = lr_schedule()

In [24]:
train_ds=ds.get_train_ds(training_files,batch_size=ds.batch_size,shuffle=True,distort=True,distort_fn=tfm.aug3)

distorting...
Finished 'get_tf_dataset_2' in 0.2412 secs
Finished 'get_tf_dataset_in_batches' in 0.2415 secs
Finished 'get_train_ds' in 0.2418 secs


In [25]:
cust = {'init_pytorch': init_pytorch}


In [26]:
model=load_model('/gdrive/MyDrive/wandb-dw/best_model12.h5',cust)

In [27]:
import numpy as np
batch_size=128
EPOCHS=40
callback_list=[Log2wandb_Callback(),lr_sched,model_cpt]
model.fit(train_ds,epochs=EPOCHS, steps_per_epoch=np.ceil(86317/batch_size), initial_epoch=20,
          validation_data=test_ds, validation_steps=np.ceil(10778/batch_size),
          callbacks=callback_list,
          verbose=1)

Epoch 21/40
epoch  21 : setting learning rate to  0.001
675/675 [==============================] - 50s 73ms/step - loss: 0.5638 - accuracy: 0.8144 - val_loss: 0.6584 - val_accuracy: 0.7780

Epoch 00021: val_accuracy improved from 0.77702 to 0.77803, saving model to /gdrive/MyDrive/wandb-dw/best_model12.h5
Epoch 22/40
epoch  22 : setting learning rate to  0.002
675/675 [==============================] - 49s 72ms/step - loss: 0.5608 - accuracy: 0.8164 - val_loss: 0.6565 - val_accuracy: 0.7772

Epoch 00022: val_accuracy did not improve from 0.77803
Epoch 23/40
epoch  23 : setting learning rate to  0.003
675/675 [==============================] - 49s 72ms/step - loss: 0.5612 - accuracy: 0.8152 - val_loss: 0.6555 - val_accuracy: 0.7775

Epoch 00023: val_accuracy did not improve from 0.77803
Epoch 24/40
epoch  24 : setting learning rate to  0.004
675/675 [==============================] - 49s 72ms/step - loss: 0.5596 - accuracy: 0.8156 - val_loss: 0.6575 - val_accuracy: 0.7762

Epoch 00024: 

In [28]:
model=load_model('/gdrive/MyDrive/wandb-dw/best_model12.h5',cust)

In [29]:
opt=SGD(lr=0.0001,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,metrics=['accuracy']
              )

In [30]:
EPOCHS=41
callback_list=[Log2wandb_Callback(),model_cpt]
model.fit(train_ds,epochs=EPOCHS, steps_per_epoch=np.ceil(86317/batch_size), initial_epoch=40,
          validation_data=test_ds, validation_steps=np.ceil(10778/batch_size),
          callbacks=callback_list,
          verbose=1)

Epoch 41/41
675/675 [==============================] - 50s 73ms/step - loss: 0.4799 - accuracy: 0.8472 - val_loss: 0.6488 - val_accuracy: 0.7800

Epoch 00041: val_accuracy improved from 0.77987 to 0.77996, saving model to /gdrive/MyDrive/wandb-dw/best_model12.h5


In [31]:
run.finish()

Epoch,40
Train Loss,0.47502
Train Acc,0.84785
Val Loss,0.64876
val_acc,0.77996
LR,0.0001
_runtime,2589
_timestamp,1619020500
_step,40


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
Train Loss,█▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
Train Acc,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████
Val Loss,█▆▇▅▆▃▅▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▃▄▁▅▃▆▅▇▇▇██████████████████████████████
LR,▁▃▅▆█▇▆▅▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
